In [49]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
from matplotlib import pyplot
import collections

In [50]:
data1 = pd.read_csv("bank-additional-full.csv",sep=";")

In [51]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
dtypes: float64(5), int64(5), object(11)
memory usa

In [52]:
data1.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [55]:
nmc_vars=["age", "campaign", "pdays", "previous", "duration", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"]
ctg_vars=["job","marital","education","default","housing","loan","contact","month","day_of_week","poutcome","y"]
ctg_vars.remove("y")
target="y"



In [56]:
from sklearn import preprocessing

In [57]:
def encode_features(df_train):
    features = ctg_vars
    
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df_train[feature])
        df_train[feature] = le.transform(df_train[feature])
    return df_train

In [58]:

data1 = encode_features(data1)

In [59]:
data1=data1.drop(ctg_vars,axis=1)

In [60]:
data1=data1.drop("pdays",axis=1)

In [61]:
nmc_vars.remove("pdays")

In [62]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
data1["y"]=[1 if i=="yes" else 0 for i in data1["y"]]

In [63]:
X = data1[[i for i in df1.columns if i != "y"]]  #independent columns
y = data1["y"]    #target column i.e price range

In [64]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)



[0.16806379 0.4253499  0.07153371 0.0258288  0.02340312 0.01927277
 0.03641806 0.18506268 0.04506718]


In [65]:
data_select=data1[feat_importances.nlargest(20).index.values]
data_select["y"]=data1["y"]
X=data_select[[i for i in data_select.columns if i != "y"]].values
y=data_select["y"].values

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X_embedded, y, test_size=0.2, random_state=1, stratify=y)

In [68]:
x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size = .1,
                                                  random_state=12)

In [69]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import  metrics
#applying somte for oversampling...
sm = SMOTE(random_state=12, ratio = 1.0)
x_train_res, y_train_res = sm.fit_sample(x_train, y_train)

C:\Users\Ashna\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [70]:
clf_rf = RandomForestClassifier(n_estimators=25, random_state=12)
clf_rf.fit(x_train_res, y_train_res)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [71]:
print('Validation Results')
print(clf_rf.score(x_val, y_val))
print(recall_score(y_val, clf_rf.predict(x_val)))

Validation Results
0.9065307113377034
0.6357615894039735


In [72]:

def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='logloss', early_stopping_rounds=early_stopping_rounds,verbose_eval=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print(metrics.classification_report(dtrain[target].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.feature_importances_).sort_values(ascending=False)
    

In [73]:
predictors = [x for x in df_select.columns if x not in [target]]
df_res=pd.DataFrame(x_train_res,columns=predictors)
df_res["y"]=y_train_res

In [74]:
#Choose all predictors and perform cross-validation to select hyperparameters
xgb1= XGBClassifier(
 learning_rate =0.1,
 n_estimators=500,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, df_res, predictors)

[0]	train-logloss:0.629161+0.000388422	test-logloss:0.629378+0.00033035
[1]	train-logloss:0.58316+0.00781182	test-logloss:0.583512+0.00746659
[2]	train-logloss:0.549806+0.0109429	test-logloss:0.550278+0.0106385
[3]	train-logloss:0.509003+0.00895277	test-logloss:0.509508+0.00871203
[4]	train-logloss:0.477025+0.00728473	test-logloss:0.47764+0.00696343
[5]	train-logloss:0.449015+0.00530669	test-logloss:0.449691+0.00576414
[6]	train-logloss:0.424393+0.0063142	test-logloss:0.425232+0.00721271
[7]	train-logloss:0.405727+0.00825108	test-logloss:0.40664+0.00919414
[8]	train-logloss:0.387394+0.00920028	test-logloss:0.388342+0.0103847
[9]	train-logloss:0.372852+0.0108454	test-logloss:0.373871+0.0121299
[10]	train-logloss:0.357915+0.0084102	test-logloss:0.359029+0.00968002
[11]	train-logloss:0.34369+0.00988901	test-logloss:0.345048+0.0111901
[12]	train-logloss:0.331222+0.00784528	test-logloss:0.332647+0.00915874
[13]	train-logloss:0.318065+0.00641933	test-logloss:0.319614+0.00780337
[14]	train-lo

[113]	train-logloss:0.129229+0.000298413	test-logloss:0.136339+0.00107743
[114]	train-logloss:0.128859+0.000324398	test-logloss:0.136017+0.00108064
[115]	train-logloss:0.128556+0.000275127	test-logloss:0.135774+0.00104707
[116]	train-logloss:0.128377+0.000238162	test-logloss:0.135614+0.00102492
[117]	train-logloss:0.127982+0.000336566	test-logloss:0.135253+0.00109488
[118]	train-logloss:0.127692+0.000371781	test-logloss:0.135029+0.00112757
[119]	train-logloss:0.127315+0.000379938	test-logloss:0.134696+0.00112811
[120]	train-logloss:0.126947+0.000344039	test-logloss:0.134383+0.00111643
[121]	train-logloss:0.12661+0.000342773	test-logloss:0.134132+0.00112294
[122]	train-logloss:0.126346+0.00032397	test-logloss:0.133926+0.00111982
[123]	train-logloss:0.125987+0.000279799	test-logloss:0.133637+0.00110502
[124]	train-logloss:0.125614+0.000332174	test-logloss:0.133322+0.00112411
[125]	train-logloss:0.125343+0.000385561	test-logloss:0.133142+0.00117077
[126]	train-logloss:0.125164+0.000453405

[225]	train-logloss:0.105097+0.000418032	test-logloss:0.117798+0.00107591
[226]	train-logloss:0.104958+0.000415951	test-logloss:0.117715+0.00105735
[227]	train-logloss:0.104787+0.000421113	test-logloss:0.117587+0.00111568
[228]	train-logloss:0.10463+0.000431686	test-logloss:0.117493+0.00119496
[229]	train-logloss:0.104534+0.000424116	test-logloss:0.117456+0.00117996
[230]	train-logloss:0.104407+0.000446693	test-logloss:0.117366+0.00115916
[231]	train-logloss:0.104254+0.000521819	test-logloss:0.117246+0.00121395
[232]	train-logloss:0.104076+0.000496322	test-logloss:0.117136+0.00119074
[233]	train-logloss:0.10397+0.000506787	test-logloss:0.117088+0.0011925
[234]	train-logloss:0.103859+0.00051015	test-logloss:0.117016+0.00116569
[235]	train-logloss:0.103704+0.000505563	test-logloss:0.116919+0.0011735
[236]	train-logloss:0.103538+0.000510168	test-logloss:0.116787+0.00121013
[237]	train-logloss:0.103366+0.000447389	test-logloss:0.116638+0.00120134
[238]	train-logloss:0.103206+0.000421198	te

[336]	train-logloss:0.0925962+0.000934832	test-logloss:0.110714+0.00124964
[337]	train-logloss:0.0925204+0.000945911	test-logloss:0.110689+0.00126879
[338]	train-logloss:0.0924512+0.000920814	test-logloss:0.110658+0.00129826
[339]	train-logloss:0.0923624+0.000896694	test-logloss:0.110649+0.00128168
[340]	train-logloss:0.0922908+0.000892326	test-logloss:0.11061+0.00128216
[341]	train-logloss:0.0922204+0.000892063	test-logloss:0.110578+0.00128357
[342]	train-logloss:0.0921378+0.00088595	test-logloss:0.110528+0.00126102
[343]	train-logloss:0.0920582+0.000856683	test-logloss:0.110502+0.00128367
[344]	train-logloss:0.0919926+0.000864679	test-logloss:0.110476+0.00129763
[345]	train-logloss:0.0919272+0.000874541	test-logloss:0.110448+0.00128016
[346]	train-logloss:0.0918494+0.000862119	test-logloss:0.110416+0.00126929
[347]	train-logloss:0.0917688+0.000861795	test-logloss:0.110401+0.0012701
[348]	train-logloss:0.0916662+0.000872512	test-logloss:0.110348+0.00127356
[349]	train-logloss:0.091589

[446]	train-logloss:0.0844602+0.000699749	test-logloss:0.107585+0.00125762
[447]	train-logloss:0.0843858+0.0006744	test-logloss:0.107536+0.00126668
[448]	train-logloss:0.0843144+0.000666282	test-logloss:0.107529+0.00125672
[449]	train-logloss:0.0842372+0.000655035	test-logloss:0.107496+0.00128405
[450]	train-logloss:0.0841948+0.000655465	test-logloss:0.107488+0.00128742
[451]	train-logloss:0.0841306+0.000659777	test-logloss:0.107467+0.00127402
[452]	train-logloss:0.084072+0.00064972	test-logloss:0.107455+0.00128445
[453]	train-logloss:0.0839824+0.000656841	test-logloss:0.107412+0.00131974
[454]	train-logloss:0.0839138+0.000656776	test-logloss:0.107377+0.00127659
[455]	train-logloss:0.083837+0.000647536	test-logloss:0.107386+0.00128009
[456]	train-logloss:0.083764+0.000635388	test-logloss:0.107358+0.00128793
[457]	train-logloss:0.0837098+0.00062173	test-logloss:0.107353+0.00129128
[458]	train-logloss:0.083645+0.000629404	test-logloss:0.107334+0.00128031
[459]	train-logloss:0.0835812+0.0

C:\Users\Ashna\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Model Report
             precision    recall  f1-score   support

          0       0.97      0.97      0.97     32882
          1       0.97      0.97      0.97     32882

avg / total       0.97      0.97      0.97     65764

AUC Score (Train): 0.996349


In [47]:
print('Validation Results')

dtest_predictions = xgb1.predict(pd.DataFrame(x_val,columns=predictors))
dtest_predprob = xgb1.predict_proba(pd.DataFrame(x_val,columns=predictors))[:,1]

print(metrics.classification_report(y_val, dtest_predictions))

Validation Results
             precision    recall  f1-score   support

          0       0.95      0.95      0.95      3666
          1       0.60      0.59      0.60       453

avg / total       0.91      0.91      0.91      4119



C:\Users\Ashna\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [48]:
Column 0,1,2 has a lot of feature importance and so can be used to minimize cost and increase profit